In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os

In [11]:
prototxtPath = os.path.sep.join([r'C:\Users\vijay\Downloads\DataSet', "deploy.prototxt"])
weightsPath = os.path.sep.join([r'C:\Users\vijay\Downloads\DataSet','weights.caffemodel'])

In [12]:

prototxtPath

'C:\\Users\\vijay\\Downloads\\DataSet\\deploy.prototxt'

In [13]:
net = cv2.dnn.readNet(prototxtPath, weightsPath)

In [14]:
model = load_model(r'C:\Users\vijay\Downloads\DataSet\mobilenet_v2.model')

In [15]:
image = cv2.imread(r'C:\Users\vijay\Downloads\DataSet\512817-3861719.jpg')

In [16]:
(h, w) = image.shape[:2]

In [17]:
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

In [18]:
net.setInput(blob)
detections = net.forward()

In [19]:
detections

array([[[[0.        , 1.        , 0.9882308 , ..., 0.13838696,
          0.7316991 , 0.6132108 ],
         [0.        , 1.        , 0.38974053, ..., 0.46923995,
          1.101469  , 1.0072013 ],
         [0.        , 1.        , 0.1830604 , ..., 0.3105046 ,
          0.85438925, 0.470085  ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [21]:
for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        (mask, withoutMask) = model.predict(face)[0]
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(image, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
        
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()